In [322]:
import matplotlib.pyplot as plt

In [323]:
import sys
sys.path.append('../')

In [166]:
%load_ext autoreload
%autoreload 1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [167]:
import numpy as np

In [4]:
%aimport log_plotter

In [168]:
from log_plotter import get_run_names_events, get_data_pth_events

In [193]:
def calc_agg_grad(grads):
    total_grad = 0.0
    for key, value in grads.items():
        total_grad += value[1][:100]
    return total_grad / len(grads)

In [300]:
def test_agg_grad(grads_real, grads_calc):
    err = (grads_real - grads_calc)
    return np.all([np.abs(err) < 1e-3])

In [334]:
patterns = ['.*runs_mnist_bsgd/test1_.*']
tags = ['grad_aa/0', 'grad_ba/0', 'model_bg/0', 'model_ag/0', 'model_aa/0', 'grad_aa/1', 'grad_ba/1', 'model_bg/1', 'model_ag/1', 'model_aa/1']
logdir = '/h/iman/Code/dist-quantization/runs'
tb_files = get_run_names_events(logdir, patterns)
log_data_ga, _ = get_data_pth_events(logdir, tb_files, tags)

In [335]:
patterns = ['.*runs_mnist_bsgd/test2_.*']
tb_files = get_run_names_events(logdir, patterns)
log_data_wa, _ = get_data_pth_events(logdir, tb_files, tags)

# Test the Gradient Aggregation

In [336]:
grads_calc = calc_agg_grad(log_data_ga['grad_ba/0'])
test_agg_grad(log_data_ga['grad_aa/0']['0'][1][:100], grads_calc)

True

In [337]:
lr = 0.01
diff = log_data_ga['model_ag/0']['2'][1][:100] - log_data_ga['model_bg/0']['2'][1][:100]
np.all(np.abs(diff + lr*log_data_ga['grad_aa/0']['2'][1][:100]) < 1e-6)

True

# Test Initial State of Models Before Aggregation

In [338]:
err = log_data_ga['model_bg/0']['0'][1][0] - log_data_wa['model_bg/0']['0'][1][0]
np.all([np.abs(err) < 1e-6])

True

# Test Weight Aggregation

In [339]:
grads_calc = calc_agg_grad(log_data_wa['model_ag/0'])
test_agg_grad(log_data_wa['model_aa/0']['0'][1][:100], grads_calc)

True

# Test Equality of Gradient in Both Mehtods

In [345]:
lr = 0.01
diff = log_data_wa['model_ag/0']['0'][1][:100] - log_data_wa['model_bg/0']['0'][1][:100]
print(np.all(np.abs(diff + lr*log_data_ga['grad_ba/0']['0'][1][:100]) < 1e-3))

True


# Compare Gradient Averaging with Weight Averaging

In [347]:
print(log_data_ga['model_ag/0']['1'][1][:100] - log_data_wa['model_aa/0']['1'][1][:100])

[ 1.90921128e-04  2.46860087e-04  2.71297991e-04  2.97062099e-04
  3.24435532e-04  3.61286104e-04  4.48316336e-04  4.81247902e-04
  4.95262444e-04  5.21384180e-04  5.43169677e-04  5.63815236e-04
  6.00151718e-04  6.40183687e-04  6.78628683e-04  7.24397600e-04
  7.44573772e-04  7.69957900e-04  8.01600516e-04  8.23639333e-04
  8.20077956e-04  8.39047134e-04  8.45745206e-04  8.14750791e-04
  8.06182623e-04  8.21284950e-04  8.40045512e-04  8.23117793e-04
  8.34688544e-04  8.07620585e-04  7.79569149e-04  7.57321715e-04
  7.27519393e-04  7.30864704e-04  7.00540841e-04  6.45816326e-04
  6.10277057e-04  5.86993992e-04  5.49517572e-04  5.50046563e-04
  5.30779362e-04  4.91954386e-04  4.47839499e-04  4.09759581e-04
  3.70658934e-04  3.20777297e-04  2.73235142e-04  2.14047730e-04
  1.68994069e-04  1.42343342e-04  6.93053007e-05 -7.00354576e-07
 -2.69189477e-05 -8.74102116e-05 -1.10179186e-04 -1.55530870e-04
 -2.07372010e-04 -2.62923539e-04 -3.45610082e-04 -4.11614776e-04
 -4.92915511e-04 -5.27508